In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import glob
import ipyparallel as ipp
import momi
import os
import pandas as pd
import PTA

from IPython.display import display

pd.set_option('display.max_columns', None)

#ipyclient = ipp.Client(cluster_id="PTA")
#print(len(ipyclient))

# For this to work dadi must be installed
easySFS = "/home/isaac/src/easySFS/easySFS.py"
!which $easySFS

/home/isaac/src/easySFS/easySFS.py


In [9]:
# Set up some directories
prefix = "/home/isaac/proj/Islands2030/PTA-GalapagosGeckos/"
analysis_dir = prefix + "analysis/"
input_dir = prefix + "vcfs/"
sfs_dir = prefix + "sfs_files/"

# Get a list of all population names
all_pops = glob.glob(input_dir + "*.txt")
all_pops = sorted([x.split("/")[-1].rsplit(".", 1)[0].split("_")[0] for x in all_pops])

all_pops

['Pbarringtonensis',
 'Pbaurii',
 'Pduncanensis',
 'Pgalap',
 'Pgorii',
 'Pleei',
 'Pmaresi-Marc',
 'Pmaresi-Sant',
 'Psimpsoni']

In [9]:
# Get easySFS preview results for all populations
preview_dict = {}
for pop in all_pops:
    print(pop)
    in_vcf = input_dir + pop + ".recode.vcf"
    pop_file = input_dir + pop + "_assigns.txt"
    preview_dict[pop] = !$easySFS -i "$in_vcf" -p "$pop_file" -a --preview

Pbarringtonensis
Pbaurii
Pduncanensis
Pgalap
Pgorii
Pleei
Pmaresi-Marc
Pmaresi-Sant
Psimpsoni


In [10]:
## Check the easySFS preview results
for k, v in preview_dict.items():
    print(k, "\n", v[-2], "\n")

Pbarringtonensis 
 (2, 10663)	(3, 15743)	(4, 19557)	(5, 21542)	(6, 24120)	(7, 23358)	(8, 25241)	(9, 21249)	(10, 22569)	(11, 14969)	(12, 15738)	(13, 6263)	(14, 6545)	 

Pbaurii 
 (2, 7858)	(3, 11722)	(4, 14961)	(5, 17708)	(6, 20375)	(7, 22647)	(8, 24987)	(9, 26738)	(10, 28830)	(11, 29875)	(12, 31749)	(13, 31764)	(14, 33425)	(15, 31871)	(16, 33295)	(17, 29510)	(18, 30665)	(19, 25179)	(20, 26058)	(21, 18516)	(22, 19099)	(23, 10830)	(24, 11139)	(25, 4600)	(26, 4720)	(27, 946)	(28, 969)	 

Pduncanensis 
 (2, 11730)	(3, 16354)	(4, 20240)	(5, 18127)	(6, 20259)	(7, 10594)	(8, 11443)	 

Pgalap 
 (2, 18238)	(3, 27259)	(4, 34254)	(5, 39962)	(6, 45259)	(7, 49850)	(8, 54289)	(9, 58008)	(10, 61888)	(11, 64879)	(12, 68343)	(13, 70161)	(14, 73283)	(15, 73604)	(16, 76418)	(17, 75037)	(18, 77555)	(19, 73900)	(20, 76113)	(21, 70026)	(22, 71925)	(23, 63550)	(24, 65128)	(25, 55108)	(26, 56377)	(27, 45333)	(28, 46304)	(29, 34444)	(30, 35137)	(31, 23526)	(32, 23972)	(33, 13835)	(34, 14085)	(35, 6233)	(36, 63

In [5]:
# Get easySFS proj results for all populations

# Pduncanensis / Pmaresi-Sant
proj = 8

for pop in all_pops:
    print(pop)
    pop_sfs_dir = sfs_dir + pop
    if not os.path.exists(pop_sfs_dir):
        os.makedirs(pop_sfs_dir)
    in_vcf = input_dir + pop + ".recode.vcf"
    pop_file = input_dir + pop + "_assigns.txt"
    !$easySFS -i $in_vcf -p $pop_file -a --proj $proj -o $pop_sfs_dir -f


Pbarringtonensis

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pbarringtonensis

Pbaurii

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pbaurii

Pduncanensis

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pduncanensis

Pgalap

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pgalap

Pgorii

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pgorii

Pleei

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pleei

Pmaresi-Marc

  Processing 1 populations - ['pop1']
  SFS files written to /home/isaac/proj/Islands2030/PTA-GalapagosGeckos/sfs_files/Pmaresi-Marc

Pmar

In [10]:
## Extract the SFS for all pops
sfs_dict = {}
for pop in all_pops:
    pop_sfs_dir = sfs_dir + pop
    # Load from the dadi formatted sfs file generated by easySFS
    sfs_file = glob.glob(pop_sfs_dir + "/dadi/*.sfs")[-1]
    #sfs_file = pop_sfs_dir + "/dadi/{}.sfs".format(pop)
    sfs_dict[pop] = momi.data.convert.sfs_from_dadi(sfs_file)
len(sfs_dict)

9

# Write full (9 pop) MSFS for expansion models

In [17]:
# msfs with sorted bins and proportional bins (not counts)
sorted_early_msfs = PTA.msfs.multiSFS(list(sfs_dict.values()), sort=True, proportions=True)
display(sorted_early_msfs.df)
sorted_early_msfs.dump(outfile=analysis_dir + "msfs/Expanding-proj8-sort-props.msfs")

# msfs with sorted bins and counts (not proportions)
sorted_early_msfs = PTA.msfs.multiSFS(list(sfs_dict.values()), sort=True, proportions=False)
display(sorted_early_msfs.df)
sorted_early_msfs.dump(outfile=analysis_dir + "msfs/Expanding-proj8-sort-counts.msfs")

,pop0,pop1,pop2,pop3,pop4,pop5,pop6,pop7,pop8
[[7_1]],0.748994,0.669777,0.654110,0.617756,0.610637,0.596941,0.593813,0.592331,0.581962
[[6_2]],0.227795,0.219819,0.219751,0.212706,0.203757,0.194032,0.190330,0.190137,0.157883
[[5_3]],0.158181,0.137826,0.129205,0.128638,0.114924,0.113530,0.108528,0.096651,0.067235
[[4_4]],0.069528,0.064843,0.061477,0.058646,0.048039,0.047568,0.047225,0.039540,0.025889


,pop0,pop1,pop2,pop3,pop4,pop5,pop6,pop7,pop8
[[7_1]],35510,31457,18714,17171,15067,12318,6795,6648,3929
[[6_2]],10322,9113,6372,5143,4382,3944,2480,2434,1285
[[5_3]],5891,4539,3745,3478,2291,1679,1472,1236,1068
[[4_4]],2563,1857,1700,1551,948,742,646,523,469


# Write MSFS grouped by recent or ancient expansion from the stairwayplot analysis

In [11]:
# Write msfs to file

# Co-expansion groups
# recent 20-80Ky sp
# ancient 100Ky+ sp

recent_pops = ["Pmaresi-Marc", "Pbarringtonensis", "Pduncanensis"]
ancient_pops = ["Pgorii", "Psimpsoni", "Pmaresi-Sant", "Pleei", "Pgalap", "Pbaurii"]

# msfs with sorted bins and proportional bins (not counts)
sorted_late_msfs = PTA.msfs.multiSFS([sfs_dict[x] for x in ancient_pops], sort=True, proportions=True)
display(sorted_late_msfs.df)
sorted_late_msfs.dump(outfile=analysis_dir + "msfs/ancient-proj8-sort-props.msfs")

# msfs with sorted bins and proportional bins (not counts)
sorted_early_msfs = PTA.msfs.multiSFS([sfs_dict[x] for x in recent_pops], sort=True, proportions=True)
display(sorted_early_msfs.df)
sorted_early_msfs.dump(outfile=analysis_dir + "msfs/recent-proj8-sort-props.msfs")

,pop0,pop1,pop2,pop3,pop4,pop5
[[7_1]],0.748994,0.669777,0.654110,0.617756,0.610637,0.592331
[[6_2]],0.227795,0.219819,0.219751,0.194032,0.190137,0.157883
[[5_3]],0.129205,0.114924,0.113530,0.108528,0.096651,0.067235
[[4_4]],0.058646,0.048039,0.047568,0.047225,0.039540,0.025889


,pop0,pop1,pop2
[[7_1]],0.596941,0.593813,0.581962
[[6_2]],0.212706,0.203757,0.190330
[[5_3]],0.158181,0.137826,0.128638
[[4_4]],0.069528,0.064843,0.061477
